In [10]:
import pandas as pd
from amplpy import AMPL, Environment

In [11]:
ampl = AMPL(Environment('/opt/ampl.linux64'))

In [12]:
ampl.reset()

In [13]:
ampl.read('./Ex--1-1a.mod')

In [14]:
ampl.readData('./Ex--1-1a.dat')

In [15]:
print(ampl.getObjective('audience_coverage').get())
print(ampl.getConstraint('Budget').get())

maximize audience_coverage:
	1800000*X['number_of_minutes_in_TV'] + 
	1e+06*X['number_of_pages_in_magazine'];
subject to Budget:
	20000*X['number_of_minutes_in_TV'] + 
	10000*X['number_of_pages_in_magazine'] <= 1e+06;


In [16]:
ampl.eval('expand audience_coverage;')
ampl.eval('expand Budget;')

maximize audience_coverage:
	1800000*X['number_of_minutes_in_TV'] + 
	1e+06*X['number_of_pages_in_magazine'];

subject to Budget:
	20000*X['number_of_minutes_in_TV'] + 
	10000*X['number_of_pages_in_magazine'] <= 1e+06;



In [17]:
ampl.eval('expand X["number_of_minutes_in_TV"];')
ampl.eval('expand X["number_of_pages_in_magazine"];')

Coefficients of X['number_of_minutes_in_TV']:
	Budget               20000
	audience_coverage  1800000

Coefficients of X['number_of_pages_in_magazine']:
	Budget             10000
	audience_coverage  1e+06



In [18]:
ampl.eval('display _nvars, _ncons;')

_nvars = 2
_ncons = 1



In [19]:
ampl.solve()

MINOS 5.51: optimal solution found.
1 iterations, objective 9.8e+07


In [20]:
print(ampl.getObjective('audience_coverage').get().value())

98000000.0


#### Shadow Price or Dual Price or Marginal Price

In [21]:
ampl.eval('display Budget;')
print(ampl.getConstraint('Budget').getValues())

Budget = 100

Budget.dual 
100.00000000000001



### Decision Variable Solution and Reduced Cost

In [22]:
X_result = list()
items = ('number_of_minutes_in_TV',
        'number_of_pages_in_magazine')
for p in items:
    X_result.append([
        p,
        ampl.getVariable('X').get(p).lb(), 
        ampl.getVariable('X').get(p).value(),
        ampl.getVariable('X').get(p).ub(),
        ampl.getVariable('X').get(p).rc()
    ])
df_X_result = pd.DataFrame(X_result)
df_X_result.rename(columns={0:'DecisionVariable_X',
                            1:'X_LowerBound',
                            2:'X_Solution',
                            3:'X_UpperBound',
                            4:'X_ReducedCosts'},
                   inplace=True)
df_X_result.set_index('DecisionVariable_X',inplace=True)
df_X_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts
DecisionVariable_X,,,,
number_of_minutes_in_TV,10.0,10.0,inf,-2.000000e+05
number_of_pages_in_magazine,0.0,80.0,inf,-1.164153e-10


In [23]:
print(ampl.getData('X'))
values = ampl.getVariable('X').getValues()
print(values)
print(values.toPandas())
print(values.toList())
print(values.toDict())

   index0    |      X      
'number_of_minutes_in_TV' |      10     
'number_of_pages_in_magazine' |      80     

   index0    |    X.val    
'number_of_minutes_in_TV' |      10     
'number_of_pages_in_magazine' |      80     

                             X.val
number_of_minutes_in_TV       10.0
number_of_pages_in_magazine   80.0
[('number_of_minutes_in_TV', 10.0), ('number_of_pages_in_magazine', 80.0)]
{'number_of_minutes_in_TV': 10.0, 'number_of_pages_in_magazine': 80.0}
